In [1]:
import os
from osgeo import gdal
import numpy as np
import rasterio
from scipy.spatial import ConvexHull

In [2]:
import rasterio
from rasterio.windows import Window
from rasterio.transform import rowcol

# AOI centroid coordinates
AOICentroid = (2042110.6, -3069083.8)

# Open the raster file
with rasterio.open("/home/tim/dentata/Sentinel2_seasonal/cvmsre_qld_m202403202405_abma2.vrt") as src:
    # Get the row and column of the centroid
    row, col = rowcol(src.transform, AOICentroid[0], AOICentroid[1])
    
    # Define the window size (100x100 pixels)
    window_size = 512
    
    # Calculate the window around the centroid
    window = Window(col - window_size // 2, row - window_size // 2, window_size, window_size)
    
    # Read the data within the window
    data = src.read(window=window)

In [ ]:
import numpy as np
from scipy.ndimage import convolve
from scipy.spatial import ConvexHull

# Define a 6x3x3 convolution kernel
kernel = np.ones((6, 3, 3))

# Initialize a list to store the convex hull volumes
convex_hullsVol = []

# Iterate over the 3D data to extract each 6x3x3 subset
for i in range(data.shape[1] - 2):
    for j in range(data.shape[2] - 2):
        subset = data[:, i:i+3, j:j+3]
        
        # Convolve the subset with the kernel
        convolved = convolve(subset, kernel, mode='constant', cval=0.0)
        
        # Calculate the convex hull for the convolved subset
        mask = convolved > 0
        points = np.argwhere(mask)
        if points.size > 0:
            print(points.shape)
            hull = ConvexHull(points)
            convex_hullsVol.append(hull.volume)
        else:
            convex_hullsVol.append(0)

# Make the convex hull volumes into a 2D array
convex_hullsVol = np.array(convex_hullsVol).reshape(data.shape[1] - 2, data.shape[2] - 2)

print(convex_hullsVol)

In [ ]:
data.shape

In [ ]:
# Plot the convex hull volumes
import matplotlib.pyplot as plt
plt.imshow(convex_hullsVol, cmap='viridis')

In [4]:
hull = ConvexHull(data)

In [ ]:
hull.volume

## Try a rios function using the segments to get the convex hull

In [ ]:
def _getSegmentHull(info, inputs, outputs, otherargs):
    
    segments = inputs.segs[0]
    pca = inputs.PCA
    
    # Get unique segment IDs
    segIDs = np.unique(segments)
            
    # Get the PCA values for each segment
    for segID in segIDs:
        if segID > 0:
            mask = segments == segID
            pca_values = pca[:,mask]
            pca_values = pca_values.T
            # Calculate the convex hull volume
            try:
                hull = ConvexHull(pca_values)
                hullVol = hull.volume
                
                otherargs.hullVols[segID] = hullVol
            except:
                otherargs.hullVols[segID] = 9999
            
        else:
            otherargs.hullVols[segID] = 9999

In [26]:
from rios import applier, cuiprogress
from scipy.spatial import ConvexHull
import numpy as np

def _getSegmentHull(info, inputs, outputs, otherargs):
    segments = inputs.segs[0]
    pca = inputs.PCA
    
    # Get unique segment IDs
    segIDs = np.unique(segments)
            
    # Get the PCA values for each segment
    for segID in segIDs:
        if segID > 0:
            mask = segments == segID
            pca_values = pca[:, mask]
            pca_values = pca_values.T
            pca_length = len(pca_values)
            
            # print(f"Segment ID: {segID}, PCA Length: {pca_length}")
            # print(f"PCA Values: {pca_values}")
            
            # Calculate the convex hull volume
            try:
                hull = ConvexHull(pca_values)
                hullVol = hull.volume
                # print(f"Hull Volume for Segment {segID}: {hullVol}")
                
                # Check if there's already a value in the dictionary
                existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
                # print(f"Existing Volume: {existing_vol}, Existing Length: {existing_length}")
                
                # Compare lengths and update dictionary
                if pca_length > existing_length:
                    # print(f"Updating Segment {segID}: New Length {pca_length}, Old Length {existing_length}")
                    otherargs.hullVols[segID] = (hullVol, pca_length)
                else:
                    otherargs.hullVols[segID] = (existing_vol, existing_length)
            except Exception as e:
                # print(f"Error calculating hull for Segment {segID}: {e}")
                # If there's an error, keep the existing value or set to 0 if not present
                existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
                otherargs.hullVols[segID] = (max(existing_vol, 0), pca_length)
        else:
            existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
            otherargs.hullVols[segID] = (max(existing_vol, 0), 0)
            
# def _getSegmentHull(info, inputs, outputs, otherargs):
    
#     segments = inputs.segs[0]
#     pca = inputs.PCA
    
#     # Get unique segment IDs
#     segIDs = np.unique(segments)
            
#     # Get the PCA values for each segment
#     for segID in segIDs:
#         if segID > 0:
#             mask = segments == segID
#             pca_values = pca[:,mask]
#             pca_values = pca_values.T
#             # Calculate the convex hull volume
#             try:
#                 hull = ConvexHull(pca_values)
#                 hullVol = hull.volume
                
#                 otherargs.hullVols[segID] = hullVol
#             except:
#                 otherargs.hullVols[segID] = 9999
            
#         else:
#             otherargs.hullVols[segID] = 9999




# Get the no data value
# ds = gdal.Open("/home/tim/dentata/Sentinel2_seasonal/cvmsre_qld_m202109202111_abma2.vrt")
# noData = ds.GetRasterBand(1).GetNoDataValue()

def getSegmentHull():    
    # Create the RIOS file objects
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    # Setup the IO
    infiles.segs = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_segs_id.tif"
    infiles.PCA = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_250m.tif"

    # Get the otherargs
    otherargs = applier.OtherInputs()
    otherargs.hullVols = {}

    # Controls for the processing   
    controls = applier.ApplierControls()
    controls.windowxsize = 512
    controls.windowysize = 512
    controls.setStatsIgnore(0) #  nodata
    controls.progress = cuiprogress.CUIProgressBar()
    controls.setFootprintType("INTERSECTION")
    controls.setResampleMethod("near")
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                    "ZLEVEL=9",
                                    "PREDICTOR=2",
                                    "BIGTIFF=YES",
                                    "TILED=YES",
                                    "INTERLEAVE=BAND",
                                    "NUM_THREADS=ALL_CPUS",
                                    "BLOCKXSIZE=512",
                                    "BLOCKYSIZE=512"])
    controls.setOverlap = 128

    # # Set concurrency depending on system
    # conc = applier.ConcurrencyStyle(numReadWorkers=3,
    #                                 numComputeWorkers=2,
    #                                 computeWorkerKind="CW_THREADS",
    #                                 )

    # controls.setConcurrencyStyle(conc)

    # Run the function
    print("Processing PCA")
    applier.apply(_getSegmentHull, infiles, outfiles, otherargs, controls=controls)
    
    return otherargs.hullVols

HullVols = getSegmentHull()   

Processing PCA


In [27]:
# Export the hull volumes to a raster
from rios import applier, cuiprogress

def _writeSegmentHull(info, inputs, outputs, otherargs):
    
    segments = inputs.segs[0]
    nodata = segments == 0
    
    outputrast = np.zeros(segments.shape, dtype=np.float32) 
       
    # Use the dictionary to write the hull volumes to the output raster
    for segID in otherargs.hullVols.keys():
        mask = segments == segID
        outputrast[mask] = otherargs.hullVols[segID][0] # Get volumes not lengths
    
    #rescale to uint16
    outputrast = (outputrast - otherargs.minVol) / (otherargs.maxVol - otherargs.minVol) *65535
    # Mask no data
    outputrast[nodata] = 0
    
    outputrast = outputrast.reshape(1, outputrast.shape[0], outputrast.shape[1])
    
    outputs.hullRast = outputrast.astype(np.uint16)




# Get the no data value
# ds = gdal.Open("/home/tim/dentata/Sentinel2_seasonal/cvmsre_qld_m202109202111_abma2.vrt")
# noData = ds.GetRasterBand(1).GetNoDataValue()

def writeSegmentHull():    
    # Create the RIOS file objects
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    # Setup the IO
    infiles.segs = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_segs_id.tif"

    outfiles.hullRast = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_segs_hullVol16_v2.tif"

    # Get the otherargs
    otherargs = applier.OtherInputs()
    otherargs.hullVols = HullVols
    values = list(HullVols.values())
    volumes = [x[0] for x in values]
    otherargs.minVol = min(volumes)
    otherargs.maxVol = max(volumes)

    # Controls for the processing   
    controls = applier.ApplierControls()
    controls.windowxsize = 512
    controls.windowysize = 512
    controls.setStatsIgnore(0) #  nodata
    controls.progress = cuiprogress.CUIProgressBar()
    controls.setFootprintType("INTERSECTION")
    controls.setResampleMethod("near")
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                    "ZLEVEL=9",
                                    "PREDICTOR=2",
                                    "BIGTIFF=YES",
                                    "TILED=YES",
                                    "INTERLEAVE=BAND",
                                    "NUM_THREADS=ALL_CPUS",
                                    "BLOCKXSIZE=512",
                                    "BLOCKYSIZE=512"])

    # Set concurrency depending on system
    # conc = applier.ConcurrencyStyle(numReadWorkers=3,
    #                                 numComputeWorkers=2,
    #                                 computeWorkerKind="CW_THREADS",
    #                                 )

    # controls.setConcurrencyStyle(conc)

    # Run the function
    print("Processing PCA")
    applier.apply(_writeSegmentHull, infiles, outfiles, otherargs, controls=controls)

writeSegmentHull()

Processing PCA

Computing Pyramid Layers...

Computing Statistics...


In [ ]:
# get range of values in HullVols
values = np.array(list(HullVols.values()))
values = values[values != 9999]
print(np.min(values), np.max(values))
scaled = (values - np.min(values)) / (np.max(values) - np.min(values)) *65534
scaled = scaled.astype(np.uint16)

In [ ]:
print(np.min(scaled), np.max(scaled))

In [25]:
HullVols

{np.uint32(0): (0, 0),
 np.uint32(1034): (2027464017.500004, 7),
 np.uint32(1085): (8698353688.833324, 10),
 np.uint32(1086): (258514266.6666657, 4),
 np.uint32(1135): (1941930658.8333395, 7),
 np.uint32(1136): (6070965847.333346, 8),
 np.uint32(1137): (4817388256.166667, 17),
 np.uint32(1138): (25195642248.500023, 16),
 np.uint32(1139): (1093880377.4999979, 5),
 np.uint32(1175): (620795961.0000018, 7),
 np.uint32(1214): (7348719370.666661, 11),
 np.uint32(1215): (451021373.3333421, 6),
 np.uint32(1274): (19282586770.66666, 13),
 np.uint32(1275): (347882332.50000304, 5),
 np.uint32(1311): (1390033294.9999888, 5),
 np.uint32(1312): (802369234.9999936, 5),
 np.uint32(1313): (3655666131.6666646, 9),
 np.uint32(1352): (3576463244.000005, 8),
 np.uint32(1391): (57236062095.3333, 8),
 np.uint32(1414): (39359983234.00004, 6),
 np.uint32(1415): (2064209661.4999988, 6),
 np.uint32(1463): (387762483.49999684, 4),
 np.uint32(1464): (32359732428.333344, 15),
 np.uint32(1465): (1638310145.333334, 1

## Function for Rao's Q